In [76]:
!pip -q install sastrawi
!pip -q install nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import random
from joblib import dump
#from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras

%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset_terbaru.csv')

In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   teks    279 non-null    object
 1   label   279 non-null    object
dtypes: object(2)
memory usage: 4.5+ KB


In [80]:
def casefolding(text):
  text = text.lower()
  text = re.sub(r'https?://\S+|www\.\S+', '', text)
  text = re.sub(r'[-+]?[0-9]+', '', text)
  text = re.sub(r'[^\w\s]','', text)
  text = text.strip()
  return text

In [81]:
raw_sample   = data['teks'].iloc[50]
case_folding = casefolding(raw_sample)

print('Raw data\t: ', raw_sample)
print('Case folding\t: ', case_folding)

Raw data	:  akses ke halaman kelas
Case folding	:  akses ke halaman kelas


In [82]:
special_stemming_rules = {
    'a': 'aaa',
    'b': 'bbb',
    'c': 'ccc',
    'jawab': 'pilih',
    'di mana': 'dimana',
    'dimana': 'mana',
    'home': 'beranda',
    'awal': 'beranda',
    'utama': 'beranda',
    'depan': 'beranda',
    'course': 'kelas',
    'raport': 'rapor',
    'rapot': 'rapor',
    'leaderboard': 'peringkat',
    'load ulang': 'refresh',
    'perbarui': 'refresh',
    'muat ulang': 'refresh',
    'reload': 'refresh',
    'segarkan': 'refresh',
    'merefresh': 'refresh',
    # tambahkan aturan khusus lainnya di sini jika diperlukan
}

In [83]:
#key_norm = pd.read_csv('key_norm.csv')

factory = StemmerFactory()
stemmer = factory.create_stemmer()

key_norm = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/key_norm.csv')  # Pastikan Anda sudah mengganti nama file sesuai dengan yang Anda gunakan

def text_normalize(text):
    # Mengganti kata-kata sesuai dengan aturan khusus
    text = ' '.join([special_stemming_rules.get(word, word) for word in text.split()])

    # Mengganti singkatan
    text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0] if (key_norm['singkat'] == word).any() else word for word in text.split()])


    # Mengonversi ke huruf kecil
    text = str.lower(text)

    return text

In [84]:
from nltk.corpus import stopwords

stopwords_ind = set(stopwords.words('indonesian'))

# Kata-kata yang ingin dikecualikan
exceptions = ['mana', 'akhir', 'mulai', 'lanjut', 'ikut', 'soal', 'tanya', 'tunjuk', 'sebut', 'jelas']

# Menghapus kata-kata tersebut dari daftar stopwords
stopwords_ind = [word for word in stopwords_ind if word not in exceptions]

# Menambahkan stopwords tambahan jika diperlukan
more_stopwords = ['ya']
stopwords_ind += more_stopwords


In [85]:
import json
import os

# Tentukan path lengkap untuk menyimpan file JSON
json_path = '/content/drive/MyDrive/Colab Notebooks/tfjs_model/stopwords_indonesia.json'

# Pastikan direktori sudah ada, jika tidak, buat
os.makedirs(os.path.dirname(json_path), exist_ok=True)

# Menyimpan data ke dalam file JSON
with open(json_path, 'w') as json_file:
    json.dump(stopwords_ind, json_file)


In [86]:
len(stopwords_ind)

748

In [87]:
# Lihat daftar stopword yang disediakan NLTK
print(stopwords_ind)

['semasih', 'walaupun', 'agaknya', 'diberi', 'dimaksud', 'sebagaimana', 'bersama-sama', 'bisa', 'bersiap-siap', 'ketika', 'saya', 'terlebih', 'lagian', 'macam', 'tak', 'empat', 'jikalau', 'kapankah', 'menjadi', 'seketika', 'kelima', 'sekaligus', 'disini', 'dapat', 'terdapat', 'walau', 'semakin', 'didapat', 'kapanpun', 'kita', 'masing', 'usah', 'ingat', 'berupa', 'tidaklah', 'perlunya', 'begitulah', 'berada', 'masihkah', 'setidaknya', 'cara', 'terus', 'jadilah', 'bagian', 'seharusnya', 'menegaskan', 'sedangkan', 'memang', 'ingin', 'ialah', 'kemungkinannya', 'depan', 'sebagainya', 'sendirinya', 'justru', 'diri', 'menambahkan', 'sebelum', 'inginkan', 'ke', 'suatu', 'waduh', 'diungkapkan', 'wah', 'waktu', 'siap', 'cukupkah', 'aku', 'semisal', 'dibuat', 'sekiranya', 'sebaik-baiknya', 'menanya', 'begitupun', 'sehingga', 'diantaranya', 'amatlah', 'selamanya', 'dimulailah', 'tetap', 'sebetulnya', 'mengingatkan', 'selanjutnya', 'tinggi', 'ibarat', 'bilakah', 'mereka', 'digunakan', 'melihat', 'a

In [88]:
def remove_stop_words(text):
  clean_words = []
  text = text.split()
  for word in text:
      if word not in stopwords_ind:
          clean_words.append(word)
  return " ".join(clean_words)

In [89]:
raw_sample       = data['teks'].iloc[5]
case_folding     = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)

print('Raw data\t\t: ', raw_sample)
print('Case folding\t\t: ', case_folding)
print('Stopword removal\t: ', stopword_removal)

Raw data		:  lihat beranda
Case folding		:  lihat beranda
Stopword removal	:  lihat beranda


In [90]:
def stemming(text):
  text = stemmer.stem(text)
  return text

In [91]:
raw_sample       = data['teks'].iloc[5]
case_folding     = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)
text_stemming    = stemming(stopword_removal)

print('Raw data\t\t: ', raw_sample)
print('Case folding\t\t: ', case_folding)
print('Stopword removal\t: ', stopword_removal)
print('Stemming\t\t: ', text_stemming)

Raw data		:  lihat beranda
Case folding		:  lihat beranda
Stopword removal	:  lihat beranda
Stemming		:  lihat beranda


In [92]:
def text_preprocessing_process(text):
  text = casefolding(text)
  print('case: ',text)
  text = text_normalize(text)
  print('norm: ',text)
  text = stemming(text)
  print('stem: ',text)
  text = remove_stop_words(text)
  print('remove: ',text)
  return text

In [93]:
%%time
data['clean_teks'] = data['teks'].apply(text_preprocessing_process)

case:  beranda
norm:  beranda
stem:  beranda
remove:  beranda
case:  utama
norm:  beranda
stem:  beranda
remove:  beranda
case:  pilih beranda
norm:  pilih beranda
stem:  pilih beranda
remove:  pilih beranda
case:  kembali ke beranda
norm:  kembali ke beranda
stem:  kembali ke beranda
remove:  beranda
case:  pergi ke halaman utama
norm:  pergi ke halaman beranda
stem:  pergi ke halaman beranda
remove:  pergi halaman beranda
case:  lihat beranda
norm:  lihat beranda
stem:  lihat beranda
remove:  lihat beranda
case:  pergi ke halaman awal
norm:  pergi ke halaman beranda
stem:  pergi ke halaman beranda
remove:  pergi halaman beranda
case:  pergi ke menu beranda
norm:  pergi ke menu beranda
stem:  pergi ke menu beranda
remove:  pergi menu beranda
case:  pergi ke home
norm:  pergi ke beranda
stem:  pergi ke beranda
remove:  pergi beranda
case:  pergi ke beranda
norm:  pergi ke beranda
stem:  pergi ke beranda
remove:  pergi beranda
case:  buka halaman utama
norm:  buka halaman beranda
stem: 

In [94]:
data

,teks,label,clean_teks
0,beranda,pergi beranda,beranda
1,utama,pergi beranda,beranda
2,pilih beranda,pergi beranda,pilih beranda
3,kembali ke beranda,pergi beranda,beranda
4,pergi ke halaman utama,pergi beranda,pergi halaman beranda
...,...,...,...
274,bacakan peringkat saya,peringkat saya,baca peringkat
275,cetak sertifikat,cetak sertifikat,cetak sertifikat
276,download sertifikat,cetak sertifikat,download sertifikat
277,unduh sertifikat,cetak sertifikat,unduh sertifikat


In [95]:
# Simpan data yang telah melalui text preprocessing agar kita tidak perlu menjalankan proses tersebut mulai awal (Opsional)
data.to_csv('clean_data.csv')

In [96]:
X = data['clean_teks']
y = data['label']

In [97]:
X

0                    beranda
1                    beranda
2              pilih beranda
3                    beranda
4      pergi halaman beranda
               ...          
274           baca peringkat
275         cetak sertifikat
276      download sertifikat
277         unduh sertifikat
278        simpan sertifikat
Name: clean_teks, Length: 279, dtype: object

In [98]:
y

0         pergi beranda
1         pergi beranda
2         pergi beranda
3         pergi beranda
4         pergi beranda
             ...       
274      peringkat saya
275    cetak sertifikat
276    cetak sertifikat
277    cetak sertifikat
278    cetak sertifikat
Name: label, Length: 279, dtype: object

In [99]:
from sklearn.preprocessing import LabelEncoder

# Inisialisasi objek LabelEncoder
label_encoder = LabelEncoder()

# Fit dan transform label unik
encoded_labels = label_encoder.fit_transform(data['label'])

# Tampilkan hasil
result_df = pd.DataFrame({
    'original_label': data['label'],
    'encoded_label': encoded_labels
})


# Tampilkan DataFrame hasil encoding
result_df

,original_label,encoded_label
0,pergi beranda,12
1,pergi beranda,12
2,pergi beranda,12
3,pergi beranda,12
4,pergi beranda,12
...,...,...
274,peringkat saya,16
275,cetak sertifikat,6
276,cetak sertifikat,6
277,cetak sertifikat,6


In [100]:
z = pd.DataFrame(encoded_labels)
z

,0
0,12
1,12
2,12
3,12
4,12
...,...
274,16
275,6
276,6
277,6


In [101]:
# Buat dictionary baru
label_dict = dict(zip(result_df['encoded_label'], result_df['original_label']))

# Urutkan dictionary berdasarkan nilai (encoded_label)
sorted_label_dict = dict(sorted(label_dict.items(), key=lambda item: item[1]))

# Tampilkan dictionary yang telah diurutkan
print(sorted_label_dict)

label_perintah = len(sorted_label_dict)


{0: 'cari kelas berjalan', 1: 'cari kelas mudah', 2: 'cari kelas normal', 3: 'cari kelas selesai', 4: 'cari kelas sulit', 5: 'cari semua kelas', 6: 'cetak sertifikat', 7: 'hentikan kuis', 8: 'jelaskan instruksi', 9: 'jumlah kelas', 10: 'kerjakan kuis', 11: 'muat ulang halaman', 12: 'pergi beranda', 13: 'pergi kelas', 14: 'pergi peringkat', 15: 'pergi rapor', 16: 'peringkat saya', 17: 'pilih a', 18: 'pilih b', 19: 'pilih c', 20: 'pilih materi sekarang', 21: 'saya dimana', 22: 'sebutkan kelas', 23: 'ulangi soal'}


In [102]:
# Konversi kamus ke JSON
json_data = json.dumps(sorted_label_dict, indent=2)

# Menyimpan JSON ke file
with open('/content/drive/MyDrive/Colab Notebooks/tfjs_model/label_encoder.json', 'w') as file:
    file.write(json_data)

In [103]:
X

0                    beranda
1                    beranda
2              pilih beranda
3                    beranda
4      pergi halaman beranda
               ...          
274           baca peringkat
275         cetak sertifikat
276      download sertifikat
277         unduh sertifikat
278        simpan sertifikat
Name: clean_teks, Length: 279, dtype: object

In [104]:
tf_idf = TfidfVectorizer(ngram_range=(1,1))
tf_idf.fit(X)

TfidfVectorizer()

In [105]:
# Melihat Jumlah Fitur
fitur = len(tf_idf.get_feature_names_out())
print(fitur)

83


In [106]:
# Melihat fitur-fitur apa saja yang ada di dalam corpus
print(tf_idf.get_feature_names_out())

['aaa' 'akhir' 'akses' 'alih' 'arah' 'baca' 'bantu' 'bbb' 'beranda'
 'beritahu' 'browser' 'buka' 'butuh' 'cari' 'ccc' 'cek' 'cetak' 'download'
 'fitur' 'go' 'halaman' 'henti' 'ikut' 'informasi' 'instruksi' 'jalan'
 'jelas' 'kambali' 'kelas' 'kerja' 'klik' 'kuis' 'kunjung' 'laman'
 'lanjut' 'layar' 'letak' 'lihat' 'load' 'lokasi' 'mana' 'masuk' 'materi'
 'menu' 'mohon' 'muat' 'mudah' 'mulai' 'navigasi' 'normal' 'open' 'opsi'
 'paham' 'pandu' 'pergi' 'peringkat' 'pertatama' 'pilih' 'pindah' 'posisi'
 'rapor' 'refresh' 'sebut' 'sedia' 'selesai' 'sertifikat' 'simpan' 'situs'
 'soal' 'sulit' 'tab' 'tampil' 'tanya' 'taut' 'tolong' 'tolonh' 'tombol'
 'total' 'tuju' 'tunjuk' 'tutup' 'ulang' 'unduh']


In [107]:
X_tf_idf = tf_idf.transform(X).toarray()
X_tf_idf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.76352516],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [108]:
# Melihat matriks jumlah token menggunakan TF IDF, lihat perbedaannya dengan metode BoW
# Data ini siap untuk dimasukkan dalam proses pemodelan (machine learning)

data_tf_idf = pd.DataFrame(X_tf_idf, columns=tf_idf.get_feature_names_out())
data_tf_idf

,aaa,akhir,akses,alih,arah,baca,bantu,bbb,beranda,beritahu,...,taut,tolong,tolonh,tombol,total,tuju,tunjuk,tutup,ulang,unduh
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.629839,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.534408,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,0.0,0.0,0.0,0.0,0.0,0.878349,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
275,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
276,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
277,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.763525


In [109]:
with open('tf_idf_feature.pickle', 'wb') as output:
  pickle.dump(X_tf_idf, output)

In [110]:
# Mengubah nilai data tabular tf-idf menjadi array agar dapat dijalankan pada proses seleksi fitur
X = np.array(data_tf_idf)
y = np.array(z)

In [111]:
chi2_features = SelectKBest(chi2, k=fitur)
X_kbest_features = chi2_features.fit_transform(X_tf_idf, y)

In [112]:
# Reduced features
print('Original feature number:', X.shape[1])
print('Reduced feature number:', X_kbest_features.shape[1])

Original feature number: 83
Reduced feature number: 83


In [113]:
# y

In [114]:
# chi2_features.scores_ adalah nilai chi-square, semakin tinggi nilainya maka semakin baik fiturnya
data_chi2 = pd.DataFrame(chi2_features.scores_, columns=['nilai'])
data_chi2

,nilai
0,217.771366
1,62.230282
2,7.162441
3,11.782321
4,10.831669
...,...
78,17.456828
79,31.713488
80,74.906006
81,106.322179


In [115]:
# Menampilkan fitur beserta nilainya
feature = tf_idf.get_feature_names_out()
data_chi2['fitur'] = feature
data_chi2

,nilai,fitur
0,217.771366,aaa
1,62.230282,akhir
2,7.162441,akses
3,11.782321,alih
4,10.831669,arah
...,...,...
78,17.456828,tuju
79,31.713488,tunjuk
80,74.906006,tutup
81,106.322179,ulang


In [116]:
# Mengurutkan fitur terbaik
sorted_data = data_chi2.sort_values(by='nilai', ascending=False)
sorted_data

,nilai,fitur
0,217.771366,aaa
7,217.771366,bbb
14,217.771366,ccc
49,192.032428,normal
69,192.032428,sulit
...,...,...
70,6.169725,tab
43,4.999135,menu
73,4.989027,taut
56,4.242145,pertatama


In [117]:
feature

array(['aaa', 'akhir', 'akses', 'alih', 'arah', 'baca', 'bantu', 'bbb',
       'beranda', 'beritahu', 'browser', 'buka', 'butuh', 'cari', 'ccc',
       'cek', 'cetak', 'download', 'fitur', 'go', 'halaman', 'henti',
       'ikut', 'informasi', 'instruksi', 'jalan', 'jelas', 'kambali',
       'kelas', 'kerja', 'klik', 'kuis', 'kunjung', 'laman', 'lanjut',
       'layar', 'letak', 'lihat', 'load', 'lokasi', 'mana', 'masuk',
       'materi', 'menu', 'mohon', 'muat', 'mudah', 'mulai', 'navigasi',
       'normal', 'open', 'opsi', 'paham', 'pandu', 'pergi', 'peringkat',
       'pertatama', 'pilih', 'pindah', 'posisi', 'rapor', 'refresh',
       'sebut', 'sedia', 'selesai', 'sertifikat', 'simpan', 'situs',
       'soal', 'sulit', 'tab', 'tampil', 'tanya', 'taut', 'tolong',
       'tolonh', 'tombol', 'total', 'tuju', 'tunjuk', 'tutup', 'ulang',
       'unduh'], dtype=object)

In [118]:
# Menampilkan mask pada feature yang diseleksi
# False berarti fitur tidak terpilih dan True berarti fitur terpilih
mask = chi2_features.get_support()
mask

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [119]:
# Menampilkan fitur-fitur terpilih berdasarkan mask atau nilai tertinggi yang sudah dikalkulasi pada Chi-Square
new_feature = []

for bool, f in zip(mask, feature):
  if bool:
    new_feature.append(f)
  selected_feature = new_feature

selected_feature

['aaa',
 'akhir',
 'akses',
 'alih',
 'arah',
 'baca',
 'bantu',
 'bbb',
 'beranda',
 'beritahu',
 'browser',
 'buka',
 'butuh',
 'cari',
 'ccc',
 'cek',
 'cetak',
 'download',
 'fitur',
 'go',
 'halaman',
 'henti',
 'ikut',
 'informasi',
 'instruksi',
 'jalan',
 'jelas',
 'kambali',
 'kelas',
 'kerja',
 'klik',
 'kuis',
 'kunjung',
 'laman',
 'lanjut',
 'layar',
 'letak',
 'lihat',
 'load',
 'lokasi',
 'mana',
 'masuk',
 'materi',
 'menu',
 'mohon',
 'muat',
 'mudah',
 'mulai',
 'navigasi',
 'normal',
 'open',
 'opsi',
 'paham',
 'pandu',
 'pergi',
 'peringkat',
 'pertatama',
 'pilih',
 'pindah',
 'posisi',
 'rapor',
 'refresh',
 'sebut',
 'sedia',
 'selesai',
 'sertifikat',
 'simpan',
 'situs',
 'soal',
 'sulit',
 'tab',
 'tampil',
 'tanya',
 'taut',
 'tolong',
 'tolonh',
 'tombol',
 'total',
 'tuju',
 'tunjuk',
 'tutup',
 'ulang',
 'unduh']

In [120]:
# Memilih fitur yang telah terseleksi dari vocabulary TF IDF
kbest_feature = {}

for (k,v) in tf_idf.vocabulary_.items():
  if k in selected_feature:
    kbest_feature[k] = v

In [121]:
tf_idf.vocabulary_.items()

dict_items([('beranda', 8), ('pilih', 57), ('pergi', 54), ('halaman', 20), ('lihat', 37), ('menu', 43), ('buka', 11), ('navigasi', 48), ('klik', 30), ('pindah', 58), ('tab', 70), ('masuk', 41), ('tampil', 71), ('situs', 67), ('kunjung', 32), ('taut', 73), ('arah', 4), ('pertatama', 56), ('akses', 2), ('laman', 33), ('kelas', 28), ('alih', 3), ('rapor', 60), ('go', 19), ('kambali', 27), ('tuju', 78), ('open', 50), ('peringkat', 55), ('mana', 40), ('posisi', 59), ('cek', 15), ('lokasi', 39), ('letak', 36), ('tolong', 74), ('informasi', 23), ('cari', 13), ('instruksi', 24), ('pandu', 53), ('fitur', 18), ('bantu', 6), ('jelas', 26), ('tunjuk', 79), ('butuh', 12), ('paham', 52), ('muat', 45), ('ulang', 81), ('refresh', 61), ('layar', 35), ('load', 38), ('browser', 10), ('tombol', 76), ('sedia', 63), ('mudah', 46), ('normal', 49), ('sulit', 69), ('sebut', 62), ('total', 77), ('mohon', 44), ('soal', 68), ('tanya', 72), ('tolonh', 75), ('aaa', 0), ('opsi', 51), ('bbb', 7), ('ccc', 14), ('kuis'

In [122]:
kbest_feature

{'beranda': 8,
 'pilih': 57,
 'pergi': 54,
 'halaman': 20,
 'lihat': 37,
 'menu': 43,
 'buka': 11,
 'navigasi': 48,
 'klik': 30,
 'pindah': 58,
 'tab': 70,
 'masuk': 41,
 'tampil': 71,
 'situs': 67,
 'kunjung': 32,
 'taut': 73,
 'arah': 4,
 'pertatama': 56,
 'akses': 2,
 'laman': 33,
 'kelas': 28,
 'alih': 3,
 'rapor': 60,
 'go': 19,
 'kambali': 27,
 'tuju': 78,
 'open': 50,
 'peringkat': 55,
 'mana': 40,
 'posisi': 59,
 'cek': 15,
 'lokasi': 39,
 'letak': 36,
 'tolong': 74,
 'informasi': 23,
 'cari': 13,
 'instruksi': 24,
 'pandu': 53,
 'fitur': 18,
 'bantu': 6,
 'jelas': 26,
 'tunjuk': 79,
 'butuh': 12,
 'paham': 52,
 'muat': 45,
 'ulang': 81,
 'refresh': 61,
 'layar': 35,
 'load': 38,
 'browser': 10,
 'tombol': 76,
 'sedia': 63,
 'mudah': 46,
 'normal': 49,
 'sulit': 69,
 'sebut': 62,
 'total': 77,
 'mohon': 44,
 'soal': 68,
 'tanya': 72,
 'tolonh': 75,
 'aaa': 0,
 'opsi': 51,
 'bbb': 7,
 'ccc': 14,
 'kuis': 31,
 'mulai': 47,
 'lanjut': 34,
 'kerja': 29,
 'henti': 21,
 'tutup': 80,


In [123]:
# Mengurutkan dictionary berdasarkan kunci (abjad)
kbest_feature_sorted = dict(sorted(kbest_feature.items()))

# Menampilkan hasil
print(kbest_feature_sorted)

{'aaa': 0, 'akhir': 1, 'akses': 2, 'alih': 3, 'arah': 4, 'baca': 5, 'bantu': 6, 'bbb': 7, 'beranda': 8, 'beritahu': 9, 'browser': 10, 'buka': 11, 'butuh': 12, 'cari': 13, 'ccc': 14, 'cek': 15, 'cetak': 16, 'download': 17, 'fitur': 18, 'go': 19, 'halaman': 20, 'henti': 21, 'ikut': 22, 'informasi': 23, 'instruksi': 24, 'jalan': 25, 'jelas': 26, 'kambali': 27, 'kelas': 28, 'kerja': 29, 'klik': 30, 'kuis': 31, 'kunjung': 32, 'laman': 33, 'lanjut': 34, 'layar': 35, 'letak': 36, 'lihat': 37, 'load': 38, 'lokasi': 39, 'mana': 40, 'masuk': 41, 'materi': 42, 'menu': 43, 'mohon': 44, 'muat': 45, 'mudah': 46, 'mulai': 47, 'navigasi': 48, 'normal': 49, 'open': 50, 'opsi': 51, 'paham': 52, 'pandu': 53, 'pergi': 54, 'peringkat': 55, 'pertatama': 56, 'pilih': 57, 'pindah': 58, 'posisi': 59, 'rapor': 60, 'refresh': 61, 'sebut': 62, 'sedia': 63, 'selesai': 64, 'sertifikat': 65, 'simpan': 66, 'situs': 67, 'soal': 68, 'sulit': 69, 'tab': 70, 'tampil': 71, 'tanya': 72, 'taut': 73, 'tolong': 74, 'tolonh': 

In [124]:
# Mengubah ke format kamus yang diinginkan
# result_dict = {row['fitur']: int(row['nilai']) for index, row in sorted_data.iterrows()}

# Mengonversi kamus ke JSON
result_json = json.dumps(kbest_feature_sorted, indent=2)

# Menyimpan JSON ke file
with open('/content/drive/MyDrive/Colab Notebooks/tfjs_model/vocab.json', 'w') as file:
    file.write(result_json)
print(result_json)

{
  "aaa": 0,
  "akhir": 1,
  "akses": 2,
  "alih": 3,
  "arah": 4,
  "baca": 5,
  "bantu": 6,
  "bbb": 7,
  "beranda": 8,
  "beritahu": 9,
  "browser": 10,
  "buka": 11,
  "butuh": 12,
  "cari": 13,
  "ccc": 14,
  "cek": 15,
  "cetak": 16,
  "download": 17,
  "fitur": 18,
  "go": 19,
  "halaman": 20,
  "henti": 21,
  "ikut": 22,
  "informasi": 23,
  "instruksi": 24,
  "jalan": 25,
  "jelas": 26,
  "kambali": 27,
  "kelas": 28,
  "kerja": 29,
  "klik": 30,
  "kuis": 31,
  "kunjung": 32,
  "laman": 33,
  "lanjut": 34,
  "layar": 35,
  "letak": 36,
  "lihat": 37,
  "load": 38,
  "lokasi": 39,
  "mana": 40,
  "masuk": 41,
  "materi": 42,
  "menu": 43,
  "mohon": 44,
  "muat": 45,
  "mudah": 46,
  "mulai": 47,
  "navigasi": 48,
  "normal": 49,
  "open": 50,
  "opsi": 51,
  "paham": 52,
  "pandu": 53,
  "pergi": 54,
  "peringkat": 55,
  "pertatama": 56,
  "pilih": 57,
  "pindah": 58,
  "posisi": 59,
  "rapor": 60,
  "refresh": 61,
  "sebut": 62,
  "sedia": 63,
  "selesai": 64,
  "sertifikat

In [125]:
kbest_feature.items()

dict_items([('beranda', 8), ('pilih', 57), ('pergi', 54), ('halaman', 20), ('lihat', 37), ('menu', 43), ('buka', 11), ('navigasi', 48), ('klik', 30), ('pindah', 58), ('tab', 70), ('masuk', 41), ('tampil', 71), ('situs', 67), ('kunjung', 32), ('taut', 73), ('arah', 4), ('pertatama', 56), ('akses', 2), ('laman', 33), ('kelas', 28), ('alih', 3), ('rapor', 60), ('go', 19), ('kambali', 27), ('tuju', 78), ('open', 50), ('peringkat', 55), ('mana', 40), ('posisi', 59), ('cek', 15), ('lokasi', 39), ('letak', 36), ('tolong', 74), ('informasi', 23), ('cari', 13), ('instruksi', 24), ('pandu', 53), ('fitur', 18), ('bantu', 6), ('jelas', 26), ('tunjuk', 79), ('butuh', 12), ('paham', 52), ('muat', 45), ('ulang', 81), ('refresh', 61), ('layar', 35), ('load', 38), ('browser', 10), ('tombol', 76), ('sedia', 63), ('mudah', 46), ('normal', 49), ('sulit', 69), ('sebut', 62), ('total', 77), ('mohon', 44), ('soal', 68), ('tanya', 72), ('tolonh', 75), ('aaa', 0), ('opsi', 51), ('bbb', 7), ('ccc', 14), ('kuis'

In [126]:
kbest_feature

{'beranda': 8,
 'pilih': 57,
 'pergi': 54,
 'halaman': 20,
 'lihat': 37,
 'menu': 43,
 'buka': 11,
 'navigasi': 48,
 'klik': 30,
 'pindah': 58,
 'tab': 70,
 'masuk': 41,
 'tampil': 71,
 'situs': 67,
 'kunjung': 32,
 'taut': 73,
 'arah': 4,
 'pertatama': 56,
 'akses': 2,
 'laman': 33,
 'kelas': 28,
 'alih': 3,
 'rapor': 60,
 'go': 19,
 'kambali': 27,
 'tuju': 78,
 'open': 50,
 'peringkat': 55,
 'mana': 40,
 'posisi': 59,
 'cek': 15,
 'lokasi': 39,
 'letak': 36,
 'tolong': 74,
 'informasi': 23,
 'cari': 13,
 'instruksi': 24,
 'pandu': 53,
 'fitur': 18,
 'bantu': 6,
 'jelas': 26,
 'tunjuk': 79,
 'butuh': 12,
 'paham': 52,
 'muat': 45,
 'ulang': 81,
 'refresh': 61,
 'layar': 35,
 'load': 38,
 'browser': 10,
 'tombol': 76,
 'sedia': 63,
 'mudah': 46,
 'normal': 49,
 'sulit': 69,
 'sebut': 62,
 'total': 77,
 'mohon': 44,
 'soal': 68,
 'tanya': 72,
 'tolonh': 75,
 'aaa': 0,
 'opsi': 51,
 'bbb': 7,
 'ccc': 14,
 'kuis': 31,
 'mulai': 47,
 'lanjut': 34,
 'kerja': 29,
 'henti': 21,
 'tutup': 80,


In [127]:
import json

# Konversi kamus ke JSON
json_data = json.dumps(kbest_feature_sorted, indent=2)




In [128]:
# Menampilkan fitur-fitur yang sudah diseleksi
# Beserta nilai vektornya pada keseluruhan data untuk dijalankan pada proses machine learning

# Hanya k fitur yang terpilih sesuai parameter k yang ditentukan sebelumnya

data_selected_feature = pd.DataFrame(X_kbest_features, columns=selected_feature)
data_selected_feature

,aaa,akhir,akses,alih,arah,baca,bantu,bbb,beranda,beritahu,...,taut,tolong,tolonh,tombol,total,tuju,tunjuk,tutup,ulang,unduh
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.629839,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.534408,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,0.0,0.0,0.0,0.0,0.0,0.878349,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
275,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
276,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
277,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.763525


In [129]:
with open('kbest_feature.pickle', 'wb') as output:
  pickle.dump(kbest_feature, output)

In [130]:
X_train, X_test, y_train, y_test = train_test_split(X_kbest_features, y, test_size=0.1, random_state=0)

In [131]:
y_train

array([[11],
       [21],
       [11],
       [13],
       [12],
       [14],
       [10],
       [23],
       [ 4],
       [ 8],
       [21],
       [13],
       [ 5],
       [11],
       [10],
       [ 0],
       [20],
       [10],
       [11],
       [14],
       [13],
       [11],
       [16],
       [21],
       [12],
       [21],
       [19],
       [ 2],
       [19],
       [12],
       [16],
       [ 4],
       [ 4],
       [12],
       [13],
       [13],
       [ 0],
       [ 3],
       [15],
       [23],
       [ 8],
       [14],
       [15],
       [13],
       [23],
       [17],
       [23],
       [13],
       [ 1],
       [12],
       [ 8],
       [17],
       [17],
       [11],
       [13],
       [ 9],
       [23],
       [ 2],
       [16],
       [ 8],
       [12],
       [15],
       [23],
       [13],
       [ 8],
       [ 0],
       [12],
       [20],
       [21],
       [12],
       [13],
       [13],
       [18],
       [ 1],
       [11],
       [14],
       [12],

In [132]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [133]:
model = keras.Sequential([
    keras.layers.Input(shape=(fitur,)),
    #keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(label_perintah, activation='softmax')
])

In [134]:
from tensorflow.keras.optimizers import Adam

# Assuming y_train and y_test are your label data
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/200
4/4 [==============================] - 1s 60ms/step - loss: 3.1898 - accuracy: 0.0518 - val_loss: 3.1575 - val_accuracy: 0.1786
Epoch 2/200
4/4 [==============================] - 0s 19ms/step - loss: 3.1428 - accuracy: 0.1554 - val_loss: 3.1150 - val_accuracy: 0.1786
Epoch 3/200
4/4 [==============================] - 0s 14ms/step - loss: 3.1002 - accuracy: 0.1873 - val_loss: 3.0720 - val_accuracy: 0.2143
Epoch 4/200
4/4 [==============================] - 0s 23ms/step - loss: 3.0532 - accuracy: 0.2351 - val_loss: 3.0247 - val_accuracy: 0.2857
Epoch 5/200
4/4 [==============================] - 0s 12ms/step - loss: 3.0038 - accuracy: 0.2669 - val_loss: 2.9725 - val_accuracy: 0.2857
Epoch 6/200
4/4 [==============================] - 0s 12ms/step - loss: 2.9478 - accuracy: 0.2590 - val_loss: 2.9136 - val_accuracy: 0.2500
Epoch 7/200
4/4 [==============================] - 0s 19ms/step - loss: 2.8846 - accuracy: 0.2430 - val_loss: 2.8473 - val_accuracy: 0.2500
Epoch 8/200
4/4 [===

In [135]:
model.save("/content/drive/MyDrive/Colab Notebooks/teks_klasifikasi.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [136]:
# Prediksi untuk Data Latih
predicted_train_probs = model.predict(X_train)
predicted_train = np.argmax(predicted_train_probs, axis=1)

8/8 [==============================] - 0s 4ms/step


In [137]:
y_train

array([[11],
       [21],
       [11],
       [13],
       [12],
       [14],
       [10],
       [23],
       [ 4],
       [ 8],
       [21],
       [13],
       [ 5],
       [11],
       [10],
       [ 0],
       [20],
       [10],
       [11],
       [14],
       [13],
       [11],
       [16],
       [21],
       [12],
       [21],
       [19],
       [ 2],
       [19],
       [12],
       [16],
       [ 4],
       [ 4],
       [12],
       [13],
       [13],
       [ 0],
       [ 3],
       [15],
       [23],
       [ 8],
       [14],
       [15],
       [13],
       [23],
       [17],
       [23],
       [13],
       [ 1],
       [12],
       [ 8],
       [17],
       [17],
       [11],
       [13],
       [ 9],
       [23],
       [ 2],
       [16],
       [ 8],
       [12],
       [15],
       [23],
       [13],
       [ 8],
       [ 0],
       [12],
       [20],
       [21],
       [12],
       [13],
       [13],
       [18],
       [ 1],
       [11],
       [14],
       [12],

In [138]:
print("Train Accuracy:", accuracy_score(y_train, predicted_train))

Train Accuracy: 0.9681274900398407


In [139]:
# Prediksi untuk Data Uji
predicted_test_probs = model.predict(X_test)
predicted_test = np.argmax(predicted_test_probs, axis=1)

1/1 [==============================] - 0s 77ms/step


In [140]:
y_test

array([[17],
       [18],
       [20],
       [12],
       [13],
       [10],
       [ 0],
       [ 5],
       [23],
       [15],
       [ 2],
       [13],
       [11],
       [ 8],
       [15],
       [14],
       [12],
       [20],
       [13],
       [11],
       [ 8],
       [12],
       [12],
       [15],
       [14],
       [13],
       [ 4],
       [12]])

In [141]:
print("Test Accuracy:", accuracy_score(y_test, predicted_test))

Test Accuracy: 1.0


In [142]:
print("\nClassification Report for Test Data:\n", classification_report(y_test, predicted_test))


Classification Report for Test Data:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         5
          13       1.00      1.00      1.00         4
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00         3
          17       1.00      1.00      1.00         1
          18       1.00      1.00      1.00         1
          20       1.00      1.00      1.00         2
          23       1.00      1.00      1.00         1

    accuracy                           1.00        28
   macro avg       1.00      1.00      1.

In [143]:
# save the model to disk
#dump(model, filename="model.json")

In [144]:
# Simpan bobot model ke file
model.save_weights("/content/drive/MyDrive/Colab Notebooks/model_weights_baru.h5")


In [145]:
from tensorflow.keras.models import load_model

# Memuat kembali model dari file
model = load_model("/content/drive/MyDrive/Colab Notebooks/teks_klasifikasi.h5")

In [146]:
print(model)

In [147]:
#@title Masukkan Teks Anda:
input_text = "saya sekarang dimana" #@param {type:"string"}

pre_input_text = text_preprocessing_process(input_text)   # lakukan text pre processing pada text input

tf_idf_vec = TfidfVectorizer(vocabulary=set(kbest_feature))       # definisikan TF_IDF
print( tf_idf_vec)
# Ubah data TF-IDF menjadi bentuk dense array
tf_idf_dense = tf_idf_vec.fit_transform([pre_input_text]).toarray()
print(tf_idf_dense)
print(type(tf_idf_dense))

result = model.predict(tf_idf_dense)  # Lakukan prediksi

# Ambil nilai argmax dari hasil prediksi
predicted_label_index = result.argmax()

print('result:', result)

# Cek apakah nilai argmax ada dalam label_dict
if np.sum(tf_idf_dense) == 0:
  print("Label tidak ditemukan dalam label_dict.")
else:
  print(label_dict[predicted_label_index], result.max())


case:  saya sekarang dimana
norm:  saya sekarang mana
stem:  saya sekarang mana
remove:  mana
TfidfVectorizer(vocabulary={'aaa', 'akhir', 'akses', 'alih', 'arah', 'baca',
                            'bantu', 'bbb', 'beranda', 'beritahu', 'browser',
                            'buka', 'butuh', 'cari', 'ccc', 'cek', 'cetak',
                            'download', 'fitur', 'go', 'halaman', 'henti',
                            'ikut', 'informasi', 'instruksi', 'jalan', 'jelas',
                            'kambali', 'kelas', 'kerja', ...})
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
<class 'numpy.ndarray'>
1/1 [==============================] - 0s 90ms/step
result: [[6.03295557e-05 4.11523069e-06 3.33682749e-07 4.92602339e-06
  4.85259250e-07 1.17264153e-06 3.83867109e-06 8.234

In [148]:
# Install TensorFlow.js Converter
!pip install tensorflowjs

In [149]:


# Import Libraries
import tensorflow as tf
from tensorflowjs import converters
from google.colab import files

# Muat Model Keras
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/teks_klasifikasi.h5')

# Tentukan path untuk menyimpan model TensorFlow.js
model_path = '/content/drive/MyDrive/Colab Notebooks/tfjs_model'

# Konversi ke TensorFlow.js
converters.save_keras_model(model, model_path)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
